# HPO Term Viewer #

In [ ]:
import numpy as np
import pandas as pd
import hpotk
from hpotk.annotations.load.hpoa import SimpleHpoaDiseaseLoader
import streamlit as st
from streamlit_tree_select import tree_select
import obonet
import networkx

In [ ]:
# Get HPO
store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()

seizure = hpo.get_term("HP:0001250")
print(seizure.name)

# from hpotk.ontology import Ontology
# from hpotk.graph import OntologyGraph

In [ ]:
# from hpotk.store import OntologyType

# # Build tree
def build_tree(hpo, term_id = "HP:0000001"):
    term = hpo.get_term(term_id)

    children = hpo.graph.get_children(term)
    children_nodes = []
    for child in children:
        node = build_tree(hpo, child.value)
        if node:
            children_nodes.append(node)
    return {
        "label": f"{term.identifier} | {term.name}",
        "value": term.name,
        "children": children_nodes
    }

# hpo_hierarchy = build_tree(hpo) # don't print this it's massive

# #print(type(hpo_hierarchy))

# def load_minimal_mondo():
#     import hpotk
#     store = hpotk.configure_ontology_store()
#     return store.load_ontology(OntologyType.MONDO)

# mondo = load_minimal_mondo()
# len(mondo)
# dir(mondo)
# #mondo_tree = [ build_tree(mondo, term_id = "MONDO:0000001") ]

# # if isinstance(hpo_hierarchy, list):
# #     for i, record in enumerate(hpo_hierarchy):
# #         if i < 5:
# #             print(record)
# #         else:
# #             break
# # else: print("Not a list")

from hpotk.model import TermId, MinimalTerm
from hpotk.ontology import MinimalOntology, create_minimal_ontology
from hpotk.graph import CsrIndexedGraphFactory
from hpotk.util import open_text_io_handle_for_reading
from hpotk.ontology.load.obographs._model import create_node, create_edge, NodeType
from hpotk.ontology.load.obographs._factory import MinimalTermFactory
import json
import re
import typing

PURL_PATTERN = re.compile(r"http://purl\.obolibrary\.org/obo/(?P<curie>(?P<prefix>\w+)_(?P<id>\w+))")

def extract_curie_from_purl(purl: str) -> typing.Optional[str]:
    matcher = PURL_PATTERN.match(purl)
    return matcher.group("curie") if matcher else None

def extract_terms_ontology(nodes, prefixes_of_interest={"MONDO"}):
    curie_to_term: dict[str, TermId] = {}
    terms: list[MinimalTerm] = []
    term_factory = MinimalTermFactory()

    for data in nodes:
        node = create_node(data)
        if not node or node.type != NodeType.CLASS:
            continue
        curie = extract_curie_from_purl(node.id)
        if not curie:
            continue
        term_id = TermId.from_curie(curie)
        if term_id.prefix not in prefixes_of_interest:
            continue
        curie_to_term[curie] = term_id
        term = term_factory.create_term(term_id, node)
        if term:
            terms.append(term)

    return curie_to_term, terms

def create_edge_list(edges, curie_to_termid):
    edge_list = []
    for data in edges:
        edge = create_edge(data)
        if edge.pred != "is_a":
            continue
        src_curie = extract_curie_from_purl(edge.sub)
        dest_curie = extract_curie_from_purl(edge.obj)
        if not src_curie or not dest_curie:
            continue
        try:
            src = curie_to_termid[src_curie]
            dest = curie_to_termid[dest_curie]
        except KeyError:
            continue
        edge_list.append((src, dest))
    return edge_list

def load_minimal_ontology(url: str, prefix: str = "MONDO") -> MinimalOntology:
    with open_text_io_handle_for_reading(url) as fh:
        doc = json.load(fh)

    obograph = doc["graphs"][0]
    id_to_term_id, terms = extract_terms_ontology(obograph["nodes"], prefixes_of_interest={prefix})
    edges = create_edge_list(obograph["edges"], id_to_term_id)
    graph = CsrIndexedGraphFactory().create_graph(edges)
    return create_minimal_ontology(graph, terms, version=None)

def load_hp():
    url = "https://purl.obolibrary.org/obo/hp.json"
    return load_minimal_ontology(url, prefix="HP")

hp = load_hp()
len(hp)

In [ ]:
# Get HPOA
hpoa_path = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2025-05-06/phenotype.hpoa'

loader = SimpleHpoaDiseaseLoader(hpo)
diseases = loader.load(hpoa_path)

In [ ]:

# @st.cache_resource()
# def load_minimal_hpo():
#     import hpotk
#     store = hpotk.configure_ontology_store()
#     return store.load_minimal_hpo()

In [ ]:
import requests
# hpoa_path = "https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2025-05-06/phenotype.hpoa"

# save_path = "../data/phenotype.hpoa.tsv"

# r = requests.get(hpoa_path)
# with open("../data/phenotype.hpoa.tsv", "wb") as f:
#     f.write(r.content)

# hpoa_df = pd.read_csv("../data/phenotype.hpoa.tsv", sep="\t", comment="#", dtype=str)
# hpoa_df.head(n=5)

# # @st.cache_data
# def load_hpoa_df():
#     url = "https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2025-05-06/phenotype.hpoa?raw=true"
#     return pd.read_csv(url, sep="\t", comment="#", dtype=str)

# df = load_hpoa_df()
# df.head()

In [ ]:
## Pretty tree viewer
st.set_page_config(layout="wide")
st.title("HPO Hierarchy Viewer")

@st.cache_resource
def load_minimal_hpo():
    import hpotk
    store = hpotk.configure_ontology_store()
    return store.load_minimal_hpo()

hpo = load_minimal_hpo()
tree = build_tree(hpo)

return_select = tree_select(tree)
st.write(return_select)

In [ ]:
# def search_hpoa(curie):
#     disease = diseases[hpotk.TermId.from_curie(curie)]

#     return(disease)

# lehigh = search_hpoa("OMIM:256000")
# print(lehigh)

In [ ]:
!streamlit run app.py